In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from scipy import optimize
import math

%matplotlib inline

In [2]:
t = pd.read_csv('../result/slope.csv', index_col=0).T
v = pd.DataFrame(pd.read_excel('../result/lac.xlsx').mean()).T
v.columns = t.columns
slope = pd.concat([t, v])
slope = slope.rename(index={0:'Lactate'})
slope = pd.concat([slope, pd.DataFrame([22, 12.5, 17], columns=['Doubling Time'] , index=slope.columns).T])

In [3]:
Lac = np.array(slope.T['Lactate']/slope.T.mean()['Lactate'])
gr = np.array([np.log(2)/i for i in slope.T['Doubling Time']])
Gr = gr/gr.mean()

In [4]:
slope

,Control,Clone A,Clone B
Glucose,0.084646,0.037859,0.014288
Glutamate,0.017335,-0.004645,0.011940
Lactate,0.636289,3.734983,1.916520
Doubling Time,22.000000,12.500000,17.000000


In [5]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [6]:
v = t.T[[
    'R00200', 'R00268', 'R00341', 'R00342', 'R00351',
    'R00405', 'R00658', 'R00703', 'R01015', 'R01049',
    'R01056', 'R01061', 'R01070', 'R01082', 'R01196', 
    'R01197', 'R01325', 'R01512', 'R01518', 'R01528',
    'R01529', 'R01641', 'R01786', 'R01830', 'R01899',
    'R01900', 'R02035', 'R02164', 'R02740', 'R04779'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00014'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00025'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [7]:
df = v.T
df['Glucose_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose_input']['C00267'] = 1

df['Oxygen_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen_input']['C00007'] = 1

df['CO2_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2_output']['C00011'] = -1

df['Lactate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate_output']['C00186'] = -1

df['Acetate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate_output']['C00033'] = -1

df['Glutamate_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glutamate_input']['C00025'] = 1

df['Ammonia_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Ammonia_input']['C00014'] = 1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['ADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['ADP_Exchange']['C00002'] = -1
df['ADP_Exchange']['C00008'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00835']['C00092'] = -1
df['R00835']['C00006'] = -1
df['R00835']['C01236'] = 1
df['R00835']['C00005'] = 1
df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

df['R00243'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00243']['C00025'] = -1
df['R00243']['C00003'] = -1
df['R00243']['C00001'] = -1

df['R00243']['C00026'] = 1
df['R00243']['C00014'] = 1
df['R00243']['C00004'] = 1
df['R00243']['C00080'] = 1

df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [8]:
BIOMASS = pd.read_csv('../csv/amino_acid.csv',index_col=0)
df_Biomass =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in BIOMASS.index:
    df_Biomass[BIOMASS.T[i]['C_num']] = BIOMASS.T[i]['flux']
    
df_Biomass['C00668'] = -0.27519
df_Biomass['C00119'] = (-0.039036) + (-0.036117) + (-0.053446)

In [9]:
df_new = pd.concat([df_Biomass.T, df], axis=1)
df_new = df_new.rename(columns={0: 'BIOMASS'})
df_new = df_new.drop([
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603'
])

In [10]:
def fba1(ox):
    glc = slope['Control']['Glucose']
    glu = slope['Control']['Glutamate']
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or 
                (i=='R02035') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or (i=='CO2_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='Lactate_output'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
            
        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=ox, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=glu, upBound=glu, cat='Continuous'))
            
        elif (i=='Ammonia_input'):
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return 100, 100

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return np.array([j['BIOMASS'][0], j['Lactate_output'][0]])

In [11]:
def fba2(ox):
    glc = slope['Clone A']['Glucose']
    glu = slope['Clone A']['Glutamate']
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or 
                (i=='R02035') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or (i=='CO2_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='Lactate_output'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
            
        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=ox, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=glu, upBound=glu, cat='Continuous'))
            
        elif (i=='Ammonia_input'):
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return 100, 100

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return np.array([j['BIOMASS'][0], j['Lactate_output'][0]])

In [12]:
def fba3(ox):
    glc = slope['Clone B']['Glucose']
    glu = slope['Clone B']['Glutamate']
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or 
                (i=='R02035') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or (i=='CO2_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='Lactate_output'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
            
        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=ox, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=glu, upBound=glu, cat='Continuous'))
            
        elif (i=='Ammonia_input'):
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return 100, 100

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return np.array([j['BIOMASS'][0], j['Lactate_output'][0]])

In [13]:
def f(x):
    x1, x2, x3 = x
    h =[
        fba1(x1), 
        fba2(x2), 
        fba3(x3)    
    ]
    
    p = pd.DataFrame(h, index=['Con', 'CloneA', 'CloneB'], columns=['Biomass', 'Lactate'])
    v = np.sum((p['Biomass']/p['Biomass'].mean() - Gr)**2) + np.sum((p['Lactate']/p['Lactate'].mean() - Lac)**2)
    
    return v

In [14]:
x0 = [0.01 for i in range(0, 3)]

In [15]:
x0

[0.01, 0.01, 0.01]

In [16]:
res1 = optimize.minimize(
    f, 
    x0,
    method='Nelder-Mead'
)

In [17]:
res1

 final_simplex: (array([[3.22671531e-06, 4.65239184e-03, 4.14529481e-06],
       [2.39027223e-06, 4.65052277e-03, 3.15582504e-06],
       [2.78944036e-06, 4.65133070e-03, 3.62001356e-06],
       [1.03610984e-06, 4.64710624e-03, 1.30966364e-06]]), array([4.03913534, 4.03917258, 4.03917588, 4.03923107]))
           fun: 4.039135342699644
       message: 'Optimization terminated successfully.'
          nfev: 185
           nit: 103
        status: 0
       success: True
             x: array([3.22671531e-06, 4.65239184e-03, 4.14529481e-06])

In [18]:
x1, x2, x3  = res1.x

In [19]:
def fba_result(glc, glu, ox):
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or (i=='R02035') or
                (i=='ATP_Exchange') or (i=='AMP_Exchange') or (i=='Glutamate_output') or (i=='CO2_output') or (i=='Ammonia_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='Lactate_output'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
            
        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=ox, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=glu, upBound=glu, cat='Continuous'))
            
        elif (i=='Ammonia_input'):
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return j

In [20]:
result = pd.concat([
    fba_result(slope['Control']['Glucose'], slope['Control']['Glutamate'], x1).T, 
    fba_result(slope['Clone A']['Glucose'], slope['Clone A']['Glutamate'], x2).T, 
    fba_result(slope['Clone B']['Glucose'], slope['Clone B']['Glutamate'], x3).T]
    , axis=1)
result.columns = ['Control', 'Clone A', 'Clone B']

In [21]:
result

,Control,Clone A,Clone B
BIOMASS,7.996744e-07,1.397058e-06,1.027325e-06
R00200,-1.866241e-01,-7.106811e-02,-4.051201e-02
R00268,0.000000e+00,4.646755e-03,0.000000e+00
R00341,1.733336e-02,-4.647638e-03,1.193774e-02
R00342,1.733415e-02,5.019071e-07,1.193876e-02
R00351,0.000000e+00,-4.646755e-03,0.000000e+00
R00405,-1.733387e-02,0.000000e+00,-1.193839e-02
R00658,1.692907e-01,7.571575e-02,2.857427e-02
R00703,-1.866237e-01,-6.642065e-02,-4.051149e-02
R01015,-8.464579e-02,-3.785859e-02,-1.428766e-02


In [22]:
result.to_csv('../result/fba_mse_glu_glc.csv')

In [23]:
result.loc['BIOMASS']/result.loc['BIOMASS'][0]

Control    1.000000
Clone A    1.747034
Clone B    1.284679
Name: BIOMASS, dtype: float64

In [24]:
result.loc['Lactate_output']/result.loc['Lactate_output'][0]

Control    1.000000
Clone A    0.355907
Clone B    0.217076
Name: Lactate_output, dtype: float64

In [25]:
Gr/Gr[0]

array([1.        , 1.76      , 1.29411765])

In [26]:
Lac/Lac[0]

array([1.        , 5.86994594, 3.0120277 ])